In [13]:
import tushare as ts
from datetime import date
import numpy as np
import pandas as pd
import pdb

riskfree_retrate = 0.05
start = "2023 1 1"
end = "2024 1 26"
# # 赛轮轮胎 vs 上证指数
ticker1 = '601058'
ticker2  = "000001"

#Get stock data
stock1 = ts.get_hist_data(ticker1)
stock2 = ts.get_hist_data(ticker2)
stock1

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
date,,,,,,,,,,,,,,
2024-01-26,11.26,11.39,11.23,11.06,373213.06,-0.10,-0.88,11.382,11.694,11.706,343308.75,284443.78,236022.68,1.19
2024-01-25,11.45,11.49,11.33,10.99,513800.00,-0.27,-2.33,11.474,11.783,11.725,324790.30,266656.72,226231.68,1.64
2024-01-24,11.55,11.60,11.60,11.44,255367.33,0.07,0.61,11.618,11.854,11.735,267809.83,253150.57,205199.43,0.82
2024-01-23,11.22,11.59,11.53,11.14,296462.75,0.31,2.76,11.698,11.857,11.730,261402.96,241640.10,199011.66,0.95
2024-01-22,11.70,11.75,11.22,11.15,277700.62,-0.47,-4.02,11.832,11.876,11.730,252703.42,230285.51,190636.26,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-03,9.40,9.59,9.36,9.25,239638.19,-0.09,-0.95,9.176,9.176,9.176,252724.92,252724.92,252724.92,0.94
2021-08-02,9.17,9.50,9.45,9.06,246290.48,0.32,3.50,9.130,9.130,9.130,255996.60,255996.60,255996.60,0.96
2021-07-30,9.10,9.20,9.13,9.00,155261.88,-0.05,-0.55,9.023,9.023,9.023,259231.97,259231.97,259231.97,0.61


In [14]:
#Create a dataframe with the adjusted close
data = pd.DataFrame({
    'stock-close' : stock1['close'], 
    'market-close': stock2['close']
    }, 
    index=stock1.index
)

# Calc the stock and market retuens by computing log(n)/log(n-1)
# REF: https://www.zhihu.com/question/30113132
# 注意我这里是日收益率
data[['stock-returns','market-returns']] = np.log(
    data[['stock-close', 'market-close']] / data[['stock-close', 'market-close']].shift(1)
)

#Drop null values
data = data.dropna()

#Generate covarience matrix
covmat = np.cov(data["stock-returns"], data["market-returns"])
print(covmat)

[[8.20812195e-04 5.51087131e-05]
 [5.51087131e-05 3.33469279e-04]]


$$
Variance(X) = E[(X - E[x])^2]
$$
方差就是随机变量的值相对于期望的波动性的加权平均

$$
Covariance(X, Y) =  E[(X - E[X])(Y - E[Y])]
$$

$$
CovMatrix(X, Y) = \begin{bmatrix}
                    Cov(X, X) & Cov(Y, X) \\
                    Cov(X, Y) & Cov(Y, Y) 
                  \end{bmatrix}

$$

基于上述 可以得到 $Cov(X, X) = Var(X)$ 我们假设X为被测股票, Y为benchmark的指数, $\beta=\frac{Cov(X,Y)}{Var(Y)}$ (详细推导见:https://gregorygundersen.com/blog/2022/03/06/capm/).
这样就能用协方差矩阵计算beta




In [15]:
beta = covmat[0,1] / covmat[1,1]
beta

0.16525874048532682

beta的本质是金融资产相对于整个市场而言的波动幅度, beta大于1, 说明X和Y关联性很稳定, 市场波动小。所以X的回报率波动也小.

CAPM可以简化为$R_x = \beta R_y + n$
- $R_x$为预期回报率
- $R_y$为市场回报率

可以通过$R_x$和$R_y$的数据代入一元线性回归，求出beta, 并验证和我们公式求出beta的相似度

In [16]:
# calculate beta from regression
beta, alpha = np.polyfit(data["market-returns"], data["stock-returns"], deg=1)
print("Beta from regression: ", beta)

# calculate expected returns
# 日收益率要 * 250
expected_return = riskfree_retrate + beta * (data["market-returns"].mean() * 250 - riskfree_retrate)
print("Expected Return: ",expected_return)

Beta from regression:  0.1652587404853268
Expected Return:  0.08336033422403064
